# Classificador AutoML - Implantação

O Classificador [Automated Machine Learning (AutoML)](https://en.wikipedia.org/wiki/Automated_machine_learning) automatiza todas as tarefas presentes na aplicação de um modelo de Machine Learning, e foi inicialmente proposto como uma solução de [Inteligência Artificial](https://en.wikipedia.org/wiki/Artificial_intelligence), para problemas reais de Machine Learning.

<img src="https://cdn.pixabay.com/photo/2017/10/24/00/39/bot-icon-2883144_1280.png" alt="autoML" width="200"/>

Na aplicação de um algoritmo de Machine Learning, vários passos são necessários, como preparação dos dados, engenharia de características, seleção de modelos, otimização de hiperparâmetros, seleção de métricas de avaliação, entre outros. O AutoML automatiza todos esses processos, apresentando resultados significativos que muitas vezes são até melhores que os apresentados por humanos na aplicação de algoritmos de Machine Learning.

Este é um componente que utiliza a biblioteca [auto-sklearn](https://github.com/automl/auto-sklearn) para obter um ou mais modelos classificadores já otimizados. <br>
O auto-sklearn é um kit de ferramentas de machine learning automatizado e um substituto para [estimator](https://scikit-learn.org/stable/glossary.html#term-estimators) do [scikit-learn](https://scikit-learn.org/stable/).

Este notebook apresenta:
- como carregar modelos e outros resultados do treinamento.
- como utilizar um modelo para fornecer predições em tempo real.

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [ ]:
%%writefile Model.py
import joblib
import numpy as np
import pandas as pd

from platiagro.pipeline import GuaranteeType


class Model(object):
    def __init__(self):
        # Carrega artefatos: estimador, etc
        artifacts = joblib.load("/tmp/data/automl-classifier.joblib")
        self.pipeline = artifacts["pipeline"]
        self.features_names_training = artifacts["columns"]
        self.columns_to_filter = artifacts["columns_to_filter"]
        self.label_encoder = artifacts["label_encoder"]
        self.method = artifacts["method"]
        self.new_columns = artifacts["new_columns"]
        self.features_after_pipeline = artifacts["features_after_pipeline"]

    def class_names(self):
        column_names = np.concatenate((self.columns_to_filter, self.new_columns))
        return column_names.tolist()

    def predict(self, X, feature_names, meta=None):
        
        df = pd.DataFrame(X)
        
        if feature_names:
            # Antes de utilizar o conjunto de dados X no modelo, reordena suas features de acordo com a ordem utilizada no treinamento
            df = pd.DataFrame(X, columns=feature_names)[self.columns_to_filter]
            X = df.to_numpy()

        # adiciona a predição ao conjunto de dados
        y_prob = self.pipeline.predict_proba(X)
        y_pred = self.pipeline.predict(X)
        y_pred = self.label_encoder.inverse_transform(y_pred)
        
        df.loc[:, self.new_columns[:-1]] = y_prob
        df.loc[:, self.new_columns[-1]] = y_pred

        return df.to_numpy()